# AutoDiff Model

In [1]:
import sys
import os

# Define the path to your folder
folder_path = f'..\\..\\..\\Models\\AutoDiffusion' 

# Add the folder to sys.path
sys.path.append(folder_path)

In [2]:
import numpy as np
#import process_edited as pce
import process_GQ as pce
import autoencoder as ae
import diffusion as diff
import TabDDPMdiff as TabDiff
import pandas as pd
import torch
import os
import time
import numpy as np

In [ ]:
# strings_set = {'diabetes','oil','yeast_ml8_dataset','creditcard_sampled','HTRU','mammography'}
# strings_set = {'diabetes','oil','creditcard_sampled','HTRU','mammography'}
strings_set = {'diabetes'}
Model = 'AutoDiff'
# dataset = 'diabetes'
metrics_list = []

for dataset in strings_set:
    
    filename = f'..\\..\\..\\Datasets\\Original Data\\{dataset}.csv'
    # Read dataframe
    print(filename)
    real_df = pd.read_csv(filename)
    #real_df = real_df.drop('url', axis=1)
    # Step 2: Inspect the data and check for class imbalance
    # Assuming the last column is the label, and the rest are features
    X = real_df.iloc[:, :-1].values  # Features
    y = real_df.iloc[:, -1].values  # Labels (binary classification)
     # Separate the minority class
    real_minortiy = real_df[y == 1]
    
    threshold = 0.01 # Threshold for mixed-type variables
    parser = pce.DataFrameParser().fit(real_minortiy, threshold)
    ################################################################################################################
    # Auto-encoder hyper-parameters
    device = 'cuda' #@param {'type':'string'}
    n_epochs = 100 #@param {'type':'integer'}
    eps = 1e-5 #@param {type:"number"}
    weight_decay = 1e-6 #@param {'type':'number'}
    maximum_learning_rate = 1e-2 #@param {'type':'number'}
    lr = 2e-4 #@param {'type':'number'}
    hidden_size = 250
    num_layers = 3
    batch_size = 50

    ds = ae.train_autoencoder(real_minortiy, hidden_size, num_layers, lr, weight_decay, n_epochs, batch_size, threshold)
    latent_features = ds[1].detach()

    ################################################################################################################
    # diffusion hyper-parameters
    diff_n_epochs = 100 #@param {'type':'integer'}
    hidden_dims = (256, 512, 1024, 512, 256) #@param {type:"raw"}
    converted_table_dim = latent_features.shape[1] #@param {'type':'integer'}
    sigma = 20  #@param {'type':'integer'} 
    num_batches_per_epoch = 50 #@param {'type':'number'}
    batch_size = 50 #@param {'type':'integer'}
    T = 100  #@param {'type':'integer'}

    score = TabDiff.train_diffusion(latent_features, T, eps, sigma, lr, \
                        num_batches_per_epoch, maximum_learning_rate, weight_decay, diff_n_epochs, batch_size)

############################################################################################################### 

    # Generate fake tabular datasets
    T = 300; N = latent_features.shape[0]; P = latent_features.shape[1]


    sample = diff.Euler_Maruyama_sampling(score, T, N, P, device)


    # duration = end_time - start_time

    # time_duration.append(duration)

    gen_output = ds[0](sample, ds[2], ds[3])
    gen_df = pce.convert_to_table(real_minortiy, gen_output, threshold)
    selected_samples = gen_df.sample(n=100, random_state=42)  # For reproducibility
    # Syn _df will be the dataset after augmentation
    syn_df = pd.concat([real_df, selected_samples], ignore_index=True)
    output_directory = os.getcwd()+f'/{string}/'
    filename = f'{Model}_{string}_Augmented.csv'

    output_file = os.path.join(output_directory, filename)
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
    import numpy as np
    import matplotlib.pyplot as plt
    from ForestDiffusion import ForestDiffusionModel
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import recall_score, f1_score
    
    augmented_df = syn_df

    # Check and print the original class distribution
    unique, counts = np.unique(y, return_counts=True)
    class_dist_before = dict(zip(unique, counts))
    print(f"Class distribution before augmentation: {class_dist_before}")# Labels (binary classification)

    X_balanced = augmented_df.iloc[:, :-1].values  # Features
    y_balanced = augmented_df.iloc[:, -1].values  # Labels (binary classification)

    # Check and print the Augmented class distribution
    unique, counts = np.unique(y_balanced, return_counts=True)
    class_dist_after = dict(zip(unique, counts))
    print(f"Class distribution after augmentation: {class_dist_after}")

    # Step 6: Split the dataset into training and test sets (original and balanced)
    X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(X, y, test_size=0.3, random_state=42)
    X_train_bal, X_test_bal, y_train_bal, y_test_bal = train_test_split(X_balanced, y_balanced, test_size=0.3, random_state=42)

    # Step 7: Train a simple classifier on both original and generated datasets
    clf_orig = RandomForestClassifier(random_state=42)
    clf_orig.fit(X_train_orig, y_train_orig)

    clf_bal = RandomForestClassifier(random_state=42)
    clf_bal.fit(X_train_bal, y_train_bal)

    # Step 8: Predict and calculate recall and F1 scores
    y_pred_orig = clf_orig.predict(X_test_orig)
    y_pred_bal = clf_bal.predict(X_test_orig)

    prec_orig = precision_score(y_test_orig, y_pred_orig)
    prec_bal = precision_score(y_test_orig, y_pred_bal)

    recall_orig = recall_score(y_test_orig, y_pred_orig)
    recalls_bal = recall_score(y_test_orig, y_pred_bal)

    f1_orig = f1_score(y_test_orig, y_pred_orig)
    f1_bal = f1_score(y_test_orig, y_pred_bal)

    # Step 9: Print and store the performance metrics
    # Store metrics in a dictionary
    metrics = {
    "Dataset": dataset,
    "Precision_Original": prec_orig,
    "Precision_Generated": prec_bal,
    "Recall_Original": recall_orig,
    "Recall_Generated": recalls_bal,
    "F1_Original": f1_orig,   
    "F1_Generated": f1_bal,
    "Num_Fake_Samples": len(augmented_df) - len(real_df),
    "Synthetic/Original_Ratio":100*(len(augmented_df) - len(real_df))/len(real_minortiy)
    }

    # Append the dictionary to the list
    metrics_list.append(metrics)

    print(f"Precision score (original data): {prec_orig:.4f}")
    print(f"Precision score (generated data): {prec_bal:.4f}")
    print(f"Recall score (original data): {recall_orig:.4f}")
    print(f"Recall score (generated data): {recalls_bal:.4f}")
    print(f"F1 score (original data): {f1_orig:.4f}")
    print(f"F1 score (generated data): {f1_bal:.4f}")
    print("Classification Report (original data):\n", classification_report(y_test_orig, y_pred_orig))
    print("Classification Report (generated data):\n", classification_report(y_test_orig, y_pred_bal))


    print(f"Number of fake samples generated: {len(augmented_df)-len(real_df)}")

..\..\..\Datasets\Original Data\diabetes.csv


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

c:\Users\Tawfique\Thesis\Data_Augmentation\Notebooks\Ongoing\AutoDiff\..\..\..\Models\AutoDiffusion\TabDDPMdiff.py:270: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t = torch.tensor(t)
c:\Users\Tawfique\Thesis\Data_Augmentation\Notebooks\Ongoing\AutoDiff\..\..\..\Models\AutoDiffusion\TabDDPMdiff.py:276: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t = torch.tensor(t)


In [ ]:
metrics_df = pd.DataFrame(metrics_list)
metrics_df.head()
# Save the DataFrame to a CSV file
metrics_df.to_csv("Auto_Diff_Forest_CM_different_datasets_metric.csv", index=False)